# 📘 Day 3: Your First ML Model - Linear Regression

**🎯 Goal:** Build, train, and evaluate a complete Machine Learning model

**⏱️ Time:** 60-75 minutes

**🌟 Why This Matters for AI:**
- Linear Regression is the foundation of modern AI
- Neural Networks are just advanced versions of regression!
- Understanding evaluation metrics helps you build better AI systems
- Same principles power recommendation systems, price prediction, and more

---

## 🤔 What is Linear Regression?

**Simple Definition:** Finding the best straight line through data points

**The Goal:** Predict a continuous number (like price, temperature, sales)

**Examples:**
- Predict house price based on size
- Predict temperature based on time
- Predict salary based on years of experience

### 📐 The Math (Don't Panic!)

**The line equation you learned in school:**
```
y = mx + b
```

**In Machine Learning:**
```
y = w₁x₁ + w₂x₂ + ... + b
```

Where:
- **y** = prediction (what we want to find)
- **x** = features (input data)
- **w** = weights (importance of each feature)
- **b** = bias (starting point)

**The model learns the best w and b values!** 🎓

## 🧠 How Does Linear Regression Learn?

**Process:**
1. Start with random weights (w) and bias (b)
2. Make predictions
3. Calculate error (how wrong are we?)
4. Adjust weights to reduce error
5. Repeat until error is minimized

**This is called "Gradient Descent"** - the backbone of ALL ML! 🚀

In [ ]:
# Install required libraries
import sys
!{sys.executable} -m pip install scikit-learn pandas numpy matplotlib seaborn --quiet

print("✅ All libraries installed!")

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set style for beautiful plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("📚 Libraries loaded successfully!")

## 🏠 Project: Predicting House Prices

**Real-World Dataset:** California Housing Prices

**Features:**
- House size (square feet)
- Number of bedrooms
- House age
- Location score

**Target:**
- House price (what we want to predict!)

Let's build a model that real estate companies use! 🏡

### Step 1: Create/Load the Dataset 📊

In [ ]:
# Create a realistic house price dataset
np.random.seed(42)

# Generate 100 houses
n_houses = 100

# Features
house_size = np.random.randint(800, 3500, n_houses)  # sq ft
bedrooms = np.random.randint(1, 6, n_houses)  # 1-5 bedrooms
house_age = np.random.randint(0, 50, n_houses)  # 0-50 years old
location_score = np.random.randint(1, 11, n_houses)  # 1-10 rating

# Create realistic price based on features
# Formula: base price + (size effect) + (bedroom effect) - (age effect) + (location effect) + noise
base_price = 100000
price = (
    base_price + 
    (house_size * 150) +  # $150 per sq ft
    (bedrooms * 20000) +  # $20k per bedroom
    (-house_age * 1000) +  # Lose $1k per year of age
    (location_score * 15000) +  # $15k per location point
    np.random.randint(-30000, 30000, n_houses)  # Random noise
)

# Create DataFrame
data = pd.DataFrame({
    'Size_SqFt': house_size,
    'Bedrooms': bedrooms,
    'Age_Years': house_age,
    'Location_Score': location_score,
    'Price': price
})

print("🏠 House Price Dataset:")
print(data.head(10))
print(f"\n📊 Total houses: {len(data)}")

### Step 2: Explore the Data 🔍

Always visualize your data before modeling!

In [ ]:
# Statistical summary
print("📊 Statistical Summary:")
print(data.describe())

In [ ]:
# Check correlations - which features are most important?
print("🔗 Correlation with Price:")
correlations = data.corr()['Price'].sort_values(ascending=False)
print(correlations)
print("\n💡 Higher correlation = More important for prediction!")

In [ ]:
# Visualize: Price vs Size
plt.figure(figsize=(10, 6))
plt.scatter(data['Size_SqFt'], data['Price'], alpha=0.6, color='blue')
plt.xlabel('House Size (Square Feet)', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.title('House Price vs Size', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📈 Notice the upward trend? Bigger houses = Higher prices!")

In [ ]:
# Visualize all features
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Size vs Price
axes[0, 0].scatter(data['Size_SqFt'], data['Price'], alpha=0.6, color='blue')
axes[0, 0].set_xlabel('Size (sq ft)')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].set_title('Size vs Price')

# Bedrooms vs Price
axes[0, 1].scatter(data['Bedrooms'], data['Price'], alpha=0.6, color='green')
axes[0, 1].set_xlabel('Bedrooms')
axes[0, 1].set_ylabel('Price ($)')
axes[0, 1].set_title('Bedrooms vs Price')

# Age vs Price
axes[1, 0].scatter(data['Age_Years'], data['Price'], alpha=0.6, color='red')
axes[1, 0].set_xlabel('Age (years)')
axes[1, 0].set_ylabel('Price ($)')
axes[1, 0].set_title('Age vs Price')

# Location vs Price
axes[1, 1].scatter(data['Location_Score'], data['Price'], alpha=0.6, color='purple')
axes[1, 1].set_xlabel('Location Score')
axes[1, 1].set_ylabel('Price ($)')
axes[1, 1].set_title('Location vs Price')

plt.tight_layout()
plt.show()

### Step 3: Prepare Data for ML 🛠️

In [ ]:
# Separate features (X) and target (y)
X = data[['Size_SqFt', 'Bedrooms', 'Age_Years', 'Location_Score']].values
y = data['Price'].values

print("📊 Features (X) shape:", X.shape)
print("🎯 Target (y) shape:", y.shape)
print("\n✅ X has 4 features for each of 100 houses")

In [ ]:
# Split into training and testing sets
# 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("🔀 Data Split Complete!")
print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print("\n💡 Model will learn from 80 houses, test on 20 unseen houses!")

### Step 4: Create and Train the Model 🏋️

**This is where the magic happens!** ✨

In [ ]:
# Create the Linear Regression model
model = LinearRegression()

print("🤖 Model created!")
print("Model type:", type(model))

In [ ]:
# Train the model (fit it to the data)
model.fit(X_train, y_train)

print("🎓 Model trained successfully!")
print("\n✨ The model has learned the relationship between features and price!")

### Step 5: Understand What the Model Learned 🧠

Let's see the weights (importance) of each feature!

In [ ]:
# Get the learned coefficients (weights)
coefficients = model.coef_
intercept = model.intercept_

feature_names = ['Size_SqFt', 'Bedrooms', 'Age_Years', 'Location_Score']

print("🔍 What the model learned:")
print(f"\nIntercept (base price): ${intercept:,.2f}")
print("\nFeature Weights:")
for name, coef in zip(feature_names, coefficients):
    print(f"  {name}: ${coef:,.2f}")

print("\n💡 Interpretation:")
print(f"  - Each extra sq ft adds ${coefficients[0]:.2f} to price")
print(f"  - Each extra bedroom adds ${coefficients[1]:,.2f} to price")
print(f"  - Each year of age changes price by ${coefficients[2]:,.2f}")
print(f"  - Each location point adds ${coefficients[3]:,.2f} to price")

### Step 6: Make Predictions 🔮

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

print("🔮 Predictions made!")
print("\n📊 First 10 predictions vs actual prices:")
print("\nPredicted\t\tActual\t\t\tDifference")
print("="*60)
for pred, actual in zip(y_pred[:10], y_test[:10]):
    diff = abs(pred - actual)
    print(f"${pred:,.2f}\t\t${actual:,.2f}\t\t${diff:,.2f}")

In [ ]:
# Visualize predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6, color='blue', s=100)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Price ($)', fontsize=12)
plt.ylabel('Predicted Price ($)', fontsize=12)
plt.title('Predicted vs Actual House Prices', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📈 Points close to the red line = Good predictions!")

### Step 7: Evaluate Model Performance 📊

**How good is our model?**

We use 3 key metrics:

**1. MAE (Mean Absolute Error)**
- Average difference between prediction and actual
- In dollars: How much are we off on average?
- **Lower is better** ✅

**2. RMSE (Root Mean Squared Error)**
- Similar to MAE but penalizes large errors more
- **Lower is better** ✅

**3. R² Score (R-squared)**
- How much variance the model explains (0 to 1)
- 1.0 = Perfect predictions
- 0.0 = Terrible predictions
- **Higher is better** ✅

In [ ]:
# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("="*60)
print("📊 MODEL PERFORMANCE REPORT")
print("="*60)
print(f"\n1. Mean Absolute Error (MAE): ${mae:,.2f}")
print(f"   → On average, predictions are off by ${mae:,.2f}")

print(f"\n2. Root Mean Squared Error (RMSE): ${rmse:,.2f}")
print(f"   → Larger errors are penalized more")

print(f"\n3. R² Score: {r2:.4f} ({r2*100:.2f}%)")
print(f"   → Model explains {r2*100:.2f}% of price variance")

print("\n" + "="*60)

# Interpretation
if r2 > 0.8:
    print("✅ EXCELLENT MODEL! R² > 0.8")
elif r2 > 0.6:
    print("👍 GOOD MODEL! R² > 0.6")
elif r2 > 0.4:
    print("⚠️ FAIR MODEL. Room for improvement.")
else:
    print("❌ POOR MODEL. Needs major improvements.")

### Step 8: Make Predictions for New Houses 🆕

In [ ]:
# New houses we want to price
new_houses = np.array([
    [2000, 3, 10, 8],   # 2000 sq ft, 3 bed, 10 years old, location score 8
    [1500, 2, 5, 6],    # 1500 sq ft, 2 bed, 5 years old, location score 6
    [3000, 4, 2, 9],    # 3000 sq ft, 4 bed, 2 years old, location score 9
])

# Predict prices
predicted_prices = model.predict(new_houses)

print("🏠 Price Predictions for New Houses:")
print("="*70)
print("\nSize\tBeds\tAge\tLocation\tPredicted Price")
print("="*70)
for house, price in zip(new_houses, predicted_prices):
    print(f"{house[0]}\t{house[1]}\t{house[2]}\t{house[3]}\t\t${price:,.2f}")
print("="*70)

## 🎯 Why This Matters for Modern AI

**The principles you just learned power EVERYTHING in AI!**

### 🤖 **Neural Networks**
- Neural networks = Many linear regressions stacked together!
- ChatGPT, DALL-E, all use the same gradient descent learning

### 🔍 **RAG Systems**
- Predict relevance scores for documents (regression!)
- Rank results before sending to LLM

### 📊 **Recommendation Systems**
- Netflix, Spotify use regression to predict ratings
- "You might rate this movie 4.5 stars" ← Regression prediction!

### 🎨 **Image Generation**
- Predict pixel values (regression for each pixel!)
- Diffusion models = Advanced regression over time

### 🤝 **Agentic AI**
- Predict value of different actions
- Choose action with highest predicted value

## 🎯 YOUR TURN: Interactive Exercise

**Challenge:** Build a model to predict student exam scores!

**Dataset:**
- Study hours per week
- Attendance percentage
- Previous exam score
- Sleep hours per night

**Target:** Final exam score (0-100)

Follow all the steps we learned!

In [ ]:
# Student exam score dataset
np.random.seed(42)
n_students = 80

study_hours = np.random.randint(5, 40, n_students)
attendance = np.random.randint(50, 100, n_students)
previous_score = np.random.randint(40, 95, n_students)
sleep_hours = np.random.randint(4, 10, n_students)

# Create realistic exam scores
exam_score = (
    20 +  # Base score
    (study_hours * 1.2) +  # More study = higher score
    (attendance * 0.3) +  # Better attendance = higher score
    (previous_score * 0.2) +  # Past performance matters
    (sleep_hours * 2) +  # Sleep is important!
    np.random.randint(-10, 10, n_students)  # Noise
)
exam_score = np.clip(exam_score, 0, 100)  # Keep scores 0-100

student_data = pd.DataFrame({
    'Study_Hours': study_hours,
    'Attendance_%': attendance,
    'Previous_Score': previous_score,
    'Sleep_Hours': sleep_hours,
    'Exam_Score': exam_score
})

print("📚 Student Exam Score Dataset:")
print(student_data.head(10))

In [ ]:
# YOUR CODE HERE!

# Step 1: Explore the data
# TODO: Print statistics and check correlations

# Step 2: Prepare X and y
# TODO: Separate features and target

# Step 3: Split data
# TODO: train_test_split with 80/20

# Step 4: Create and train model
# TODO: LinearRegression and fit

# Step 5: Make predictions
# TODO: Predict on test set

# Step 6: Evaluate model
# TODO: Calculate MAE, RMSE, R²

# Step 7: Predict for a new student:
# Study 25 hours/week, 85% attendance, previous score 75, sleeps 7 hours
# TODO: Make prediction

print("Complete the TODOs above!")

### ✅ Solution (Try on your own first!)

In [ ]:
# SOLUTION
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Step 1: Explore
print("📊 Correlations with Exam Score:")
print(student_data.corr()['Exam_Score'].sort_values(ascending=False))

# Step 2: Prepare X and y
X_students = student_data[['Study_Hours', 'Attendance_%', 'Previous_Score', 'Sleep_Hours']].values
y_students = student_data['Exam_Score'].values

# Step 3: Split
X_train, X_test, y_train, y_test = train_test_split(
    X_students, y_students, test_size=0.2, random_state=42
)

# Step 4: Train
student_model = LinearRegression()
student_model.fit(X_train, y_train)
print("\n✅ Model trained!")

# Step 5: Predict
y_pred = student_model.predict(X_test)

# Step 6: Evaluate
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n📊 MODEL PERFORMANCE:")
print(f"MAE: {mae:.2f} points")
print(f"RMSE: {rmse:.2f} points")
print(f"R² Score: {r2:.4f} ({r2*100:.2f}%)")

# Step 7: Predict for new student
new_student = np.array([[25, 85, 75, 7]])  # Study, Attendance, Previous, Sleep
predicted_score = student_model.predict(new_student)

print(f"\n🎓 Predicted exam score for new student: {predicted_score[0]:.1f}/100")

# Show feature importance
print("\n💡 Feature Importance (Coefficients):")
features = ['Study_Hours', 'Attendance_%', 'Previous_Score', 'Sleep_Hours']
for feature, coef in zip(features, student_model.coef_):
    print(f"  {feature}: {coef:.2f}")

## 📋 Linear Regression Checklist

**Before building a regression model:**

**1. Understand the Problem** 🎯
- [ ] Is this a regression problem? (Predicting a number?)
- [ ] Do I have enough data? (At least 50+ samples)
- [ ] Are features related to target?

**2. Prepare Data** 🛠️
- [ ] Handle missing values
- [ ] Encode categorical variables
- [ ] Check for outliers
- [ ] Split train/test

**3. Build Model** 🏗️
- [ ] Create LinearRegression()
- [ ] Fit on training data
- [ ] Make predictions

**4. Evaluate** 📊
- [ ] Calculate MAE, RMSE, R²
- [ ] Visualize predictions vs actual
- [ ] Check if R² > 0.6 (good model)

**5. Interpret** 🧠
- [ ] Understand coefficients
- [ ] Identify most important features
- [ ] Make predictions for new data

## 🎉 Congratulations!

**You just built a complete Machine Learning model!** 🎊

**What you learned:**
- ✅ What Linear Regression is and how it works
- ✅ How to train a model using scikit-learn
- ✅ How to make predictions on new data
- ✅ How to evaluate model performance (MAE, RMSE, R²)
- ✅ How to interpret coefficients and feature importance
- ✅ Real-world application: House price prediction

**🎯 Practice Projects (Highly Recommended!):**

**1. Beginner:**
- Predict car prices based on age, mileage, brand
- Predict temperature based on time and location

**2. Intermediate:**
- Download Kaggle "House Prices" dataset
- Build a salary prediction model

**3. Advanced:**
- Predict stock prices (challenging!)
- Build a recommendation system score predictor

---

**📚 What's Next?**

**Week 7 Preview:**
- Classification models (Logistic Regression, Decision Trees)
- More advanced evaluation metrics
- Cross-validation and hyperparameter tuning

---

**💬 Key Takeaway:**

*You just built the same type of model that powers real estate predictions, salary estimations, and even parts of modern AI systems. Linear Regression is simple, but it's the foundation of EVERYTHING in machine learning. Master this, and you'll understand 80% of ML!* 🚀

---

**🔗 Connections to Modern AI:**

- **GPT/LLMs**: Predict next word using advanced regression
- **RAG Systems**: Score document relevance (regression!)
- **Recommendation Systems**: Predict ratings using regression
- **Image AI**: Predict pixel values (regression for each pixel)
- **Agentic AI**: Predict action values using regression
- **Transformers**: Attention scores computed via learned weights (regression!)

**Every advanced AI model builds on these foundations!** 🌟